<a href="https://colab.research.google.com/github/CameronKenworthyCode/python/blob/main/slippi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

I'm always working with the large (>5GB) data on a Gdrive, so first things first, mount the drive, so this assumes you have a G-drive with replay files in a folder, I provide a link to the post-wrangled csv file later. But this is part of the prep work for the project

In [ ]:
!pip install py-slippi
import pandas as pd
import numpy as np
import slippi as slp

!pip install -U -q PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import drive
from oauth2client.client import GoogleCredentials


auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
gdrive = GoogleDrive(gauth)


local_download_path = os.path.expanduser('~/data')
try:
  os.makedirs(local_download_path)
except: pass

drive.mount('/content/drive/')

file_list = gdrive.ListFile(
    {'q': "'1Nqg7wW2bV5d7vzj1YbasIaLkByn5WZYI' in parents"}).GetList() 

match_names = [file_list[i]['title'] for i in range(len(file_list))]

##BUILDING THE MASTER DATAFRAME FROM 25,000 matches

In [ ]:
def get_characters_played(match):
  test_str = str(match.start.players)
  substart = 'CSSCharacter.'
  subend = 'costume='
  #find the indexes of the strings immediately before and after the character names
  #then adjust the indexes to just give us the char names.
  start = [i.start() for i in re.finditer(substart, test_str)]
  start[0] += 13
  start[1] += 13
  end  = [i.start() for i in re.finditer(subend, test_str)]
  end[0] += -2
  end[1] += -2
  char1 = test_str[start[0]: end[0]]
  char2 = test_str[start[1]: end[1]]
  return char1, char2

In [ ]:
#this takes 4 hours

active_ports = []
list_of_stages = []
list_of_characters = []
list_of_match_lengths = list([])

for index in range(len(match_names)):
  current_match_object = slp.Game('/content/drive/My Drive/slippiDB/' + match_names[index])
  list_of_characters += [get_characters_played(current_match_object)]
  print(get_characters_played(current_match_object))
  list_of_stages.append(str(current_match_object.start.stage)[6:])
  print(str(current_match_object.start.stage)[6:])
  active_ports.append([current_match_object.start.players[i] is not None for i in range(4)])
  print(index)
  list_of_match_lengths.append(len(current_match_object.frames))
  print(len(current_match_object.frames))
  del current_match_object

df = pd.DataFrame()
df['stage'] = pd.Series(list_of_stages)
df['characters'] = pd.Series(list_of_characters)
df['file_names'] = pd.Series(match_names)
df['length_in_frames'] = pd.Series(list_of_match_lengths)
df['active_ports'] = pd.Series(active_ports)

used_ports = [] #manual mapping for ease grabbing x,y from frames
for i in range(len(df)):
  if df.active_ports[i] == [True, True, False, False]:
    used_ports += [[0,1]]
  if df.active_ports[i] == [True, False, True, False]:
    used_ports += [[0,2]]
  if df.active_ports[i] == [True, False, False, True]:
    used_ports += [[0,3]]
  if df.active_ports[i] == [False, True, True, False]:
    used_ports += [[1,2]]
  if df.active_ports[i] == [False, True, False, True]:
    used_ports += [[1,3]]
  if df.active_ports[i] == [False, False, True, True]:
    used_ports += [[2,3]]

df['active_ports'] = pd.Series(used_ports)

df.to_csv('slippiDB.csv')
!cp slippiDB.csv "drive/My Drive/"

##FINDING DESIRED SUBSET OF DATA

link to slippiDB.csv https://drive.google.com/open?id=1VR7ucC5Tf31uoXlJsb4cWWaevD7y4Tzw

In [ ]:
#You fell asleep and your runtime disconnected
df = pd.read_csv('drive/My Drive/slippiDB.csv', index_col=0)

from ast import literal_eval
for index in range(len(df)):
  df.active_ports[index] = literal_eval(df.active_ports[index])
  df.characters[index] = literal_eval(df.characters[index])

In [ ]:
#make two conditionals to sort down to only fox vs fox on battlefield
is_fox_ditto = np.array([df.characters[i][0] == 'FOX' and df.characters[i][1] == 'FOX' for i in range(len(df))])
is_battlefield = np.array([df.stage[i] == 'BATTLEFIELD' for i in range(len(df))])
data = df[is_fox_ditto & is_battlefield] #data is the new working dataframe

In [ ]:
#look at ultimate size of dataset referenced by data dataframe, should be about 4 million frames, useful for calculating when you're going to be waiting

sum_of_frames = 0

for i in data.index:
  sum_of_frames += data['length_in_frames'][i]

sum_of_frames 

##BUILD X_FEATURE MATRIX AND Y_TARGET VECTOR

In [ ]:
#functions we'll need to wrangle that data!
def get_ports(match):
  used_ports = []
  active_ports = [match.start.players[i] is not None for i in range(4)]
  if active_ports == [True, True, False, False]:
    used_ports = [0,1]
  if active_ports == [True, False, True, False]:
   used_ports = [0,2]
  if active_ports == [True, False, False, True]:
    used_ports = [0,3]
  if active_ports == [False, True, True, False]:
    used_ports = [1,2]
  if active_ports == [False, True, False, True]:
    used_ports = [1,3]
  if active_ports == [False, False, True, True]:
    used_ports = [2,3]
  return used_ports

def get_buttons(logical):
  logical_string = str(logical)
  inputs =      {'START':0,
                 'Y':0,
                 'X':0,                
                 'B':0,
                 'A':0,
                 'L':0,
                 'R':0,
                 'Z':0,
                 'NONE':0,
                }
  if logical_string.find('NONE') > -1:
    inputs['NONE'] = 1
    return inputs
  if logical_string.find('|A') > -1 or logical_string.find('.A') > -1:
    inputs['A'] = 1
  if logical_string.find('|B') > -1 or logical_string.find('.B') > -1:
    inputs['B'] = 1
  if logical_string.find('|Y') > -1 or logical_string.find('.Y') > -1:
    inputs['Y'] = 1
  if logical_string.find('|X') > -1 or logical_string.find('.X') > -1:
    inputs['X'] = 1
  if logical_string.find('|L') > -1 or logical_string.find('.L') > -1:
    inputs['L'] = 1
  if logical_string.find('|R') > -1 or logical_string.find('.R') > -1:
    inputs['R'] = 1
  if logical_string.find('|Z') > -1 or logical_string.find('.Z') > -1:
    inputs['Z'] = 1
  #if logical_string.find('CSTICK_RIGHT') > -1:
  #  inputs['CSTICK_RIGHT'] = 1
  #if logical_string.find('CSTICK_LEFT') > -1:
  #  inputs['CSTICK_LEFT'] = 1
  #if logical_string.find('CSTICK_DOWN') > -1:
  #  inputs['CSTICK_DOWN'] = 1
  #if logical_string.find('CSTICK_UP') > -1:
  #  inputs['CSTICK_UP'] = 1
  #if logical_string.find('JOYSTICK_RIGHT') > -1:
  #  inputs['JOYSTICK_RIGHT'] = 1
  #if logical_string.find('JOYSTICK_LEFT') > -1:
  #  inputs['JOYSTICK_LEFT'] = 1
  #if logical_string.find('JOYSTICK_DOWN') > -1:
  #  inputs['JOYSTICK_DOWN'] = 1
  #if logical_string.find('JOYSTICK_UP') > -1:
  #  inputs['JOYSTICK_UP'] = 1
  return  inputs

def to_y_vector_df(match_number, frame_number, frame, portslist):

  ydf = pd.DataFrame()
  ydf['frame'] = pd.Series([frame_number-1])
  ydf['match'] = pd.Series([match_number])

  p2_post = frame.ports[portslist[1]].leader.post
  p2_state = p2_post.state.name
  p2_state_age = int(np.round(p2_post.state_age))
  p2_post_state = str(p2_state + ':' + str(p2_state_age))
 
  ydf['p2_post_state'] = pd.Series([p2_post_state])

  return ydf

def to_x_matrix_df(match_number, frame_number, frame, portslist):

  xdf = pd.DataFrame()
  xdf['frame'] = pd.Series([frame_number])
  xdf['match'] = pd.Series([match_number])

  p1_pre = frame.ports[portslist[0]].leader.pre
  p2_pre = frame.ports[portslist[1]].leader.pre
  p1_post = frame.ports[portslist[0]].leader.post
  p2_post = frame.ports[portslist[1]].leader.post

  p1_buttons = get_buttons(p1_pre.buttons.logical)
  xdf['p1_A'] = pd.Series([p1_buttons['A']])
  xdf['p1_B'] = pd.Series([p1_buttons['B']])
  xdf['p1_X'] = p1_buttons['X']
  xdf['p1_Y'] = p1_buttons['Y']
  xdf['p1_L'] = p1_buttons['L']
  xdf['p1_R'] = p1_buttons['R']


  #get p1 pre message
  xdf['p1_pre_direction'] = p1_pre.direction.value
  xdf['p1_cstick_x'] = p1_pre.cstick.x
  xdf['p1_cstick_y'] = p1_pre.cstick.y
  xdf['p1_jstick_x'] = p1_pre.joystick.x
  xdf['p1_jstick_y'] = p1_pre.joystick.y
  xdf['p1_L_trigger'] = p1_pre.triggers.physical.l
  xdf['p1_R_trigger'] = p1_pre.triggers.physical.r
  xdf['p1_pre_damage'] = p1_pre.damage[0]
  xdf['p1_x_pos'] = p1_pre.position.x
  xdf['p1_y_pos'] = p1_pre.position.y
  xdf['p1_pre_state'] = p1_pre.state.name

  p2_buttons = get_buttons(p2_pre.buttons.logical)
  xdf['p2_A'] = p2_buttons['A']
  xdf['p2_B'] = p2_buttons['B']
  xdf['p2_X'] = p2_buttons['X']
  xdf['p2_Y'] = p2_buttons['Y']
  xdf['p2_L'] = p2_buttons['L']
  xdf['p2_R'] = p2_buttons['R']


  #get p2 pre message
  xdf['p2_pre_direction'] = p2_pre.direction.value
  xdf['p2_cstick_x'] = p2_pre.cstick.x
  xdf['p2_cstick_y'] = p2_pre.cstick.y
  xdf['p2_jstick_x'] = p2_pre.joystick.x
  xdf['p2_jstick_y'] = p2_pre.joystick.y
  xdf['p2_L_trigger'] = p2_pre.triggers.physical.l
  xdf['p2_R_trigger'] = p2_pre.triggers.physical.r
  xdf['p2_pre_damage'] = p2_pre.damage[0]
  xdf['p2_pre_x_pos'] = p2_pre.position.x
  xdf['p2_pre_y_pos'] = p2_pre.position.y
  xdf['p2_pre_state'] = p2_pre.state.name

  #get p1 post message
  xdf['p1_is_airborne'] = p1_post.airborne
  xdf['p1_post_damage'] = p1_post.damage
  xdf['p1_post_direction'] = p1_post.direction.value
  xdf['p1_hitstun'] = int(np.floor(p1_post.hit_stun))
  xdf['p1_jumps_remaining'] = p1_post.jumps
  if p1_post.l_cancel is not None:
    if p1_post.l_cancel.value is 2:
      xdf['p1_Lcancel_failed'] = 1
    else:
      xdf['p1_Lcancel_failed'] = 0
  else:
    xdf['p1_Lcancel_failed'] = 0
  xdf['p1_post_x_pos'] = p1_post.position.x
  xdf['p1_post_y_pos'] = p1_post.position.y
  xdf['p1_shield'] = p1_post.shield
  p1_state = p1_post.state.name
  p1_state_age = int(np.round(p1_post.state_age))
  p1_post_state = str(p1_state + ':' + str(p1_state_age))
  xdf['p1_post_state'] = p1_post_state
  xdf['p1_stocks'] = p1_post.stocks

  #get p2 post message
  xdf['p2_is_airborne'] = p2_post.airborne
  xdf['p2_post_damage'] = p2_post.damage
  xdf['p2_post_direction'] = p2_post.direction.value
  xdf['p2_hitstun'] = int(np.floor(p2_post.hit_stun))
  xdf['p2_jumps_remaining'] = p2_post.jumps
  if p2_post.l_cancel is not None:
    if p2_post.l_cancel.value is 2:
      xdf['p2_Lcancel_failed'] = 1
    else:
      xdf['p2_Lcancel_failed'] = 0
  else:
    xdf['p2_Lcancel_failed'] = 0
  xdf['p2_post_x_pos'] = p2_post.position.x
  xdf['p2_post_y_pos'] = p2_post.position.y
  xdf['p2_shield'] = p2_post.shield
  p2_state = p2_post.state.name
  p2_state_age = int(np.round(p2_post.state_age))
  p2_post_state = str(p2_state + ':' + str(p2_state_age))
  xdf['p2_post_state'] = p2_post_state
  xdf['p2_stocks'] = p2_post.stocks

  return xdf

In [ ]:
frames = []
target_frames = []

for index in range(0,len(data)): 
  #using your generated data dataframe, go through, load each match, and pull out all the features I think are useful for the matrix
  #also grab the y_target vector
  current_match = slp.Game('/content/drive/My Drive/slippiDB/' + data['file_names'][data.index[index]]) #this assumes you have the slippi database on a google drive
  
  for i in range(1, len(current_match.frames), 2):
    newframe = to_y_vector_df(index, i, current_match.frames[i], get_ports(current_match))
    print(index, i)
    target_frames += [newframe]

  for i in range(0, len(current_match.frames)-1, 2):
    newframe = to_x_matrix_df(index, i, current_match.frames[i], get_ports(current_match))
    print(index, i)
    frames += [newframe]

x_feature_matrix = pd.concat(frames) #TAKES DOUBLE FOREVER
y_target_vector = pd.concat(target_frames) #TAKES FOREVER
x_feature_matrix.set_index(['match','frame'], inplace=True)
y_target_vector.set_index(['match','frame'], inplace=True)

x_feature_matrix.to_csv('X_matrix.csv')
!cp X_matrix.csv "drive/My Drive/"

y_target_vector.to_csv('Y_vector.csv')
!cp Y_vector.csv "drive/My Drive/"

URLs for my generated files since github wont let you upload anything over 25MB

https://drive.google.com/open?id=1W5QjBp1llr2znj1bG2CgFKQcuKoOqbOo #X_Matrix

https://drive.google.com/open?id=1LTosyLQzCe_cxL8bJ3LfBwDvrwM4Dfx7 #Y_Vector

In [ ]:
#It's been 30 hours, your colab session has expired
Xdata = pd.read_csv('drive/My Drive/X_matrix.csv', index_col=[0, 1])
ydata  = pd.read_csv('drive/My Drive/Y_target.csv', index_col=[0, 1])

#sync up the stratified matrix and vector
Xdata = Xdata.reset_index()
ydata = ydata.reset_index()
Xdata = Xdata.drop('match', axis=1)
ydata = ydata.drop(['match', 'frame'], axis=1)
Xdata['p2_next_frame_post_state'] = ydata['p2_post_state'] 

#split into your train and val out of the testing data, your testing data is huge
from sklearn.model_selection import train_test_split
test, train = train_test_split(Xdata, test_size=0.05, random_state=19)
test, val = train_test_split(test, test_size=.1, random_state=19)
print(len(train), len(val))

#get the data ready to be fed into models
target = 'p2_next_frame_post_state'
X_train = train.drop(columns=target)
y_train = train[target]
X_val = val.drop(columns=target)
y_val = val[target]
#best stay away from test, it's too big
X_test = test.drop(columns=target)
y_test = test[target]

##TRAIN MODEL

In [ ]:
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import make_pipeline
import numpy as np

pipeline = make_pipeline(
    ce.OrdinalEncoder(), 
    DecisionTreeClassifier()
)

pipeline.fit(X_train, y_train)

print('Accuracy:', pipeline.score(X_val, y_val)) #it might run out of RAM here especially if train was split > .3, maybe put this in another cell

In [ ]:
#save the results in case you crash and lose everything
y_pred = pipeline.predict(X_val)
model_results = pd.DataFrame()
model_results['predicted'] = pd.Series(y_pred)
model_results['true'] = pd.Series(y_val.values)
model_results.to_csv('results.csv')
!cp results.csv "drive/My Drive/model results/"

since I'm recompiling this colab from A LOT of different colabs I don't have the model itself, though you can reproduce it. I did save the results of DecisionTree-Basic on .05X of the data https://drive.google.com/open?id=1G_cOw6bJNl_0u1adthTluqB5GfVBhBgc if you load this as a dataframe it has predicted and True columns for about 60,000 frames

##GET FEATURE IMPORTANCE

In [ ]:
#assuming you still have the model loaded

import eli5
from eli5.sklearn import PermutationImportance

dt = DecisionTreeClassifier()
encoder = ce.OrdinalEncoder()

X_train_transformed = encoder.fit_transform(X_train)
X_val_transformed = encoder.transform(X_val)

dt.fit(X_train_transformed, y_train)

permuter = PermutationImportance(
    dt, 
    scoring='accuracy', 
    n_iter=5, #might run out of RAM, you can reduce
    random_state=21
)

permuter.fit(X_val_transformed, y_val)

feature_names = X_test.columns.tolist()

importances = pd.Series(permuter.feature_importances_, feature_names).sort_values()

eli5.show_weights(
    permuter, 
    top=None, 
    feature_names=feature_names
)

##FEATURE ENGINEERING
sorry I lost my code for adding distance and angle to the X_matrix, but it's very simple loop of going to each frame, pulling the p1 and p2 x,y coordinates, and doing pythagorean theorem and then finding the angle, but here is the voronoi generator assuming you have the data df loaded from above. Also I added the code for grabbing the frame in the match in above already

In [ ]:
#make empty lists to be filled with coordinates
fox_pos = []

pos = {'FOX':fox_pos #just so we can use keys to pass around the lists of coordinates}

#GET ALL COORDINATE PAIRS

for index in range(len(df)):
  current_game_object = slp.Game('/content/drive/My Drive/slippi DB/' + data.file_names[index])
  print(index, ' :opening file: ', data.file_names[index])
  print(data.characters[index][0], '...getting frames')
  pos[data.characters[index][0]] += [current_game_object.frames[frame].ports[data.active_ports[index][0]].leader.post.position for frame in range(data.length_in_frames[index])] #put p1s coordinates in correct pos list by key
  print(data.characters[index][1], '...getting frames\n')
  pos[data.characters[index][1]] += [current_game_object.frames[frame].ports[data.active_ports[index][1]].leader.post.position for frame in range(data.length_in_frames[index])] #put p2s corrdinates in correct pos list by key

In [ ]:
all_points_df = pd.DataFrame()
all_points_df['x'] = pd.Series([all_points[i].x for i in range(len(all_points))])
all_points_df['y'] = pd.Series([all_points[i].y for i in range(len(all_points))])
all_points_df.to_csv('ALL_POINTS.csv')
!cp ALL_POINTS.csv "drive/My Drive/"

In [ ]:
kmeans = KMeans(n_clusters=12)
clusters = kmeans.fit_predict(all_points_df)
centers = kmeans.cluster_centers_
Xdata['P2_Zone'] = pd.Series(clusters)
